In [70]:
import pandas as pd

In [71]:
def cleanData(df,substance):
    df = df.drop(columns=['historianTagnummer'])
    df.rename(columns={'hstWaarde': substance + 'Value'}, inplace=True)
    df[substance+'Value'] = df[substance + 'Value'].astype('float')
    df['datumBeginMeting'] = pd.to_datetime(df['datumBeginMeting'])
    df['StartDate'] = df['datumBeginMeting'].dt.date
    df['StartTime'] = df['datumBeginMeting'].dt.time
    df['datumEindeMeting'] = pd.to_datetime(df['datumEindeMeting'])
    df['endDate'] = df['datumEindeMeting'].dt.date
    df['endTime'] = df['datumEindeMeting'].dt.time
    df = df.drop(columns=['datumBeginMeting','datumEindeMeting'])
    df.to_parquet('../data/cleanedData/'+ substance +'.parquet')
    pass

In [72]:
data_files = [
    ('../data/Ammonium_measurements.parquet', 'ammonium'),
    ('../data/Nitrate_measurements.parquet', 'nitrate'),
    ('../data/Oxygen_A.parquet', 'oxygenA'),
    ('../data/Oxygen_B.parquet', 'oxygenB'),
    ('../data/Phosphate_measurements.parquet', 'phosphate')
]

for file_path, var_name in data_files:
    df = pd.read_parquet(file_path)
    cleanData(df, var_name)

In [73]:
ammonium = pd.read_parquet('../data/cleanedData/ammonium.parquet')
ammonium.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525469 entries, 0 to 44639
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ammoniumValue  525469 non-null  object
 1   StartDate      525469 non-null  object
 2   StartTime      525469 non-null  object
 3   endDate        525469 non-null  object
 4   endTime        525469 non-null  object
dtypes: object(5)
memory usage: 24.1+ MB


In [74]:
ammonium.describe()

,ammoniumValue,StartDate,StartTime,endDate,endTime
count,525469,525469,525469,525469,525469
unique,17419,365,1440,366,1440
top,0.682,2021-10-31,00:00:00,2021-10-31,00:01:00
freq,142,1500,365,1500,365
